In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd

In [4]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
# Lets define some constants to help us later on

In [5]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

573/573 [==============================] - 0s 0us/step


In [13]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [14]:
train_y = train.pop("Species")
test_y = test.pop("Species")

In [15]:
print(train_y)

0      2
1      1
2      2
3      0
4      0
      ..
115    1
116    1
117    0
118    0
119    1
Name: Species, Length: 120, dtype: int64


In [18]:
train_y.shape

(120,)

In [38]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [39]:
my_feature_columns =[]

for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [48]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

In [49]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)

A newer approach is to use Sequential model rather than the estimator

In [50]:
# classifier = tf.keras.Sequential([
#     tf.keras.layers.DenseFeatures(my_feature_columns),
#     tf.keras.layers.Dense(30, activation='relu'),
#     tf.keras.layers.Dense(10, activation='relu'),
#     tf.keras.layers.Dense(3, activation='softmax')
# ])

# # Compile the model
# classifier.compile(optimizer='adam',
#                    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#                    metrics=['accuracy'])

In [51]:
# classifier.fit(
#     x=input_fn(train, train_y, training=True),
#     steps_per_epoch=5000  # Replace with the desired number of training steps
# )

In [52]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

Instructions for updating:
Use tf.keras instead.



Test set accuracy: 0.933



Time for Predictions

In [60]:
def input_fn(features,batch_size =256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): 
      valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn = lambda: input_fn(predict))

for predict_dict in predictions:
  class_ids = predict_dict["class_ids"][0]
  probability = predict_dict["probabilities"][class_ids]

  print("Prediction is {} . ({:.1f}%)".format(SPECIES[class_ids], probability *100))

Please type numeric values as prompted.
SepalLength: 2.2
SepalWidth: 3.4
PetalLength: 5.2
PetalWidth: 3.4


Exception ignored in: <generator object Estimator.predict at 0x7f42aafc3610>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorflow_estimator/python/estimator/estimator.py", line 618, in predict
  File "/usr/lib/python3.10/contextlib.py", line 153, in __exit__
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/ops.py", line 5931, in get_controller
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/context.py", line 272, in pop
IndexError: pop from empty list


Prediction is Virginica . (88.2%)
